<a href="https://colab.research.google.com/github/Luca-wirthner/2024_Geoinformatique_I/blob/main/pop_empl_ze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import geopandas as gpd
import pyogrio as pgr
import matplotlib.pyplot as plt
import numpy as np

Les fichiers en entrée de ce traitement de données peuvent être téléchargés directement depuis le disque dur de l'ordinateur à l'aide du code suivant:


```
from google.colab import files
files.upload()
```
Cependant, l'exécution peut prendre beaucoup de temps. Il est donc préférable de télécharger les fichiers en entrée depuis l'onglet "Files" de google colab, à l'aide de "Upload to session storage".

Les fichiers que nous devons télécharger sont respectivement:


1.   "geoinf-proj-ze-3860 (1).gpkg", renommé 'zone_etude.gpkg', qui contient la couche 'ze_3860', la zone d'étude.
2.   "admin.gpkg", qui contient la couche 'communes', où se trouvent les limites des communes et les informations liées.
3. "ofs_geostat (1).gpkg", renommé 'stat_ofs.gpkg', qui contient la couche 'pop_2023' où se trouvent les données de population sous forme de points, ainsi que la couche 'empl_2022' où se trouvent les données d'emplois sous forme de points.

Après l'import des fichiers geopackages, la commande suivante permet de vérifier le nom des couches:


```
import pyogrio as pgr
pgr.list_layers('filename')
```








In [2]:
zone_etude = gpd.read_file('zone_etude.gpkg', layer = 'ze_3860')

In [3]:
# Création des 3 périmètres
buff_500m = zone_etude.buffer(500).union_all()
buff_1km_dif = zone_etude.buffer(1000).union_all() - zone_etude.buffer(500).union_all()
buff_5km_dif = zone_etude.buffer(5000).union_all() - zone_etude.buffer(1000).union_all()

In [4]:
# Mise en dictionnaire et assignation du SRS EPSG:2056
buffs = {'500m': buff_500m, '1km': buff_1km_dif, '5km': buff_5km_dif}
gdf_buffs = { k: gpd.GeoDataFrame(geometry=[poly], crs='EPSG:2056') for k, poly in buffs.items()}

In [5]:
communes = gpd.read_file('admin.gpkg', layer = 'communes')

In [6]:
# Inclusion des polygones des communes dans les 3 périmètres
gdf_buffs_cmns = { k: gpd.overlay(gdf_buffs[k], communes, how = 'intersection') for k in gdf_buffs}

In [7]:
population = gpd.read_file('stat_ofs.gpkg', layer = 'pop_2023')
emplois = gpd.read_file('stat_ofs.gpkg', layer = 'empl_2022')

Cette fonction a été fournie par les enseignants du cours de géoinformatique dans le cadre duquel ce projet a été réalisé.

In [8]:
def sjoin_summarize(poly_lyr, pts_lyr, pts_colname, poly_newcolname):
    """
    poly_lyr:         la couche des polygones
    pts_lyr:          la couche des points
    pts_colname:      la colonne sur laquelle il faut faire la somme
    poly_newcolname:  le nom de la colonne à créer dans la couche
                      des polygones
    """
    # Faire d'abord une jointure spatiale pour associer chaque point à un polygone
    sjoin = pts_lyr.sjoin(poly_lyr, how='inner', predicate='within')

    # Faire une opération «group by» selon l'index du polygone, et calculer la somme
    somme_par_poly = sjoin.groupby(sjoin.index_right)[pts_colname].sum()

    # Enregistrer le résultat dans la couche des polygones
    poly_lyr[poly_newcolname] = poly_lyr.index.map(somme_par_poly)

In [9]:
# Jointure des périmètres maillés par les polygones des communes aux points des emplois et de la population, sommés par polygone.
for k in gdf_buffs_cmns:
    sjoin_summarize(gdf_buffs_cmns[k], population, 'BBTOT', 'pop_2023_sum')
    sjoin_summarize(gdf_buffs_cmns[k], emplois, 'B08EMPT', 'empl_2022_sum')
    gdf_buffs_cmns[k]['peri'] = k

In [10]:
# Extraction des colonnes à garder
data_500m = gdf_buffs_cmns['500m'][['bfs_nummer', 'name', 'peri', 'pop_2023_sum', 'empl_2022_sum']]
data_1km = gdf_buffs_cmns['1km'][['bfs_nummer', 'name', 'peri', 'pop_2023_sum', 'empl_2022_sum']]
data_5km = gdf_buffs_cmns['5km'][['bfs_nummer', 'name', 'peri', 'pop_2023_sum', 'empl_2022_sum']]

In [11]:
# Concaténation des 3 geodataframes en un dataframe
tabl = pd.concat([data_500m, data_1km, data_5km], ignore_index=True)

,bfs_nummer,name,peri,pop_2023_sum,empl_2022_sum
0,5938,Yverdon-les-Bains,500m,3049.0,5097.0
1,5926,Pomy,500m,68.0,88.0
2,5938,Yverdon-les-Bains,1km,4973.0,1330.0
3,5926,Pomy,1km,46.0,30.0
4,5934,Valeyres-sous-Ursins,1km,NaN,NaN
5,5930,Suscévaz,5km,36.0,4.0
6,5939,Yvonand,5km,9.0,4.0
7,5520,Essertines-sur-Yverdon,5km,450.0,92.0
8,5932,Ursins,5km,240.0,41.0
9,5561,Grandson,5km,2586.0,1042.0


In [12]:
# Remplacement des valeurs 'NaN' par 0
personnes_par_peri_et_cmne = tabl.replace(np.nan,0)

##Indices calculés pour les communes et les périmètres:



1.   Njsem: Nombre de personnes présentes pendant un jour de semaine
2.   Nnsem: Nombre de personnes présentes pendant une nuit de semaine
3.   Njwe: Nombre de personnes présentes pendant un jour de weekend
4.   Nnwe: Nombre de personnes présentes pendant une nuit de weekend

##Formules utilisées (selon les indications de l'OPAM):



1.   Njsem = 0.3 x P + 0.8 x E
2.   Nnsem = 0.9 x P + 0.05 X E
3.   Njwe = 0.6 X P + 0.05 x E
4.   Nnwe = P

Où  P représente les habitants et E les emplois





In [14]:
# Calcul des indices et ajouts en colonnes
personnes_par_peri_et_cmne['Njsem'] = ((0.3*personnes_par_peri_et_cmne['pop_2023_sum']) + (0.8*personnes_par_peri_et_cmne['empl_2022_sum']))
personnes_par_peri_et_cmne['Nnsem'] = ((0.9*personnes_par_peri_et_cmne['pop_2023_sum']) + (0.05*personnes_par_peri_et_cmne['empl_2022_sum']))
personnes_par_peri_et_cmne['Njwe'] = ((0.6*personnes_par_peri_et_cmne['pop_2023_sum']) + (0.05*personnes_par_peri_et_cmne['empl_2022_sum']))
personnes_par_peri_et_cmne['Nnwe'] = ['pop_2023_sum']

In [16]:
# Regroupement des données par périmètre
personnes_par_peri = personnes_par_peri_et_cmne[['peri','Njsem','Nnsem','Njwe','Nnwe']].groupby('peri', as_index = False).sum()

In [18]:
# Regroupement par commune
personnes_par_cmne = personnes_par_peri_et_cmne[['bfs_nummer','name','Njsem','Nnsem','Njwe','Nnwe']].groupby(['bfs_nummer','name'], as_index = False).sum()

In [37]:
# Chargement des dataframes en fichiers CSV
personnes_par_peri_et_cmne.to_csv('personnes_par_peri_et_cmne.csv', index =False)
personnes_par_peri.to_csv('personnes_par_peri.csv', index = False)
personnes_par_cmne.to_csv('personnes_par_cmne.csv', index = False)

In [38]:
# Export des fichiers vers les téléchargements de l'ordinateur
from google.colab import files
files.download('personnes_par_peri_et_cmne.csv')
files.download('personnes_par_peri.csv')
files.download('personnes_par_cmne.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>